In [1]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json

import cv2
import torch
from tqdm import tqdm_notebook
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
import torchvision

/home/briancy2/.conda/envs/iwildcam/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
META_DIR = "data/metadata/metadata/"
TRAIN_DIR = "data/train/train/"
TEST_DIR = "data/test/test/"

test_data = json.load(open(META_DIR + 'iwildcam2022_test_information.json'))
train_data = json.load(open(META_DIR + 'iwildcam2022_train_annotations.json'))

#Test images
df_test = pd.DataFrame({'id': [item['id'] for item in test_data['images']],
                                'seq_id': [item['seq_id'] for item in test_data['images']],
                                'file_name': [item['file_name'] for item in test_data['images']],
                                'location': [item['location'] for item in test_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in test_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in test_data['images']]})
#Train images
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['images']],
                                'seq_id': [item['seq_id'] for item in train_data['images']],
                                'file_name': [item['file_name'] for item in train_data['images']],
                                'location': [item['location'] for item in train_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in train_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in train_data['images']]})
# Detection for train test
detections = json.load(open(META_DIR+"iwildcam2022_mdv4_detections.json"))['images']
det_dict = dict()
for detection in tqdm(detections):
    det_dict[detection['file']] = detection['detections']
df_detection = pd.DataFrame({'file': [item['file'] for item in detections],
                                'detections': [item['detections'] for item in detections]})
# Test sequence ids
test_sequence_ids = pd.unique(df_test['seq_id'])
# Train sequence id and count
train_seq_count = pd.read_csv(META_DIR+"train_sequence_counts.csv")

100%|██████████| 261428/261428 [00:00<00:00, 1491130.13it/s]


In [28]:
from PIL import Image, ImageFile
from deep_sort.deep_sort.ocsort import OCSort

ImageFile.LOAD_TRUNCATED_IMAGES = True
file = open('submission_oc_det0.4.txt', 'w')
file.write("Id,Predicted")
counts = []
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    #print("######################### NEW SEQ ##################")
    #print(seq_row["seq_id"])
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    tracker = OCSort(det_thresh=0.4, max_age=30, min_hits=3, 
        iou_threshold=0, delta_t=3, asso_func="iou", inertia=0.2, use_byte=False)
    #results = []
    outputs = []
    for index, img_row in img_rows.iterrows():
        image = Image.open(TRAIN_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        dets = np.zeros((1, 4))
        if detections:
            dets = np.array([d["bbox"] for d in detections])
        cates = np.array([d["category"] for d in detections])
        confs = np.array([d["conf"] for d in detections])
        output = tracker.update_public(dets, cates, confs)
        #print("detections", detections)
        #print("return", output)
        if not output.size == 0:
            outputs.append(output[:,4].astype('float64').max())
        
    count = len(tracker.trackers)
    if not len(outputs) == 0:
        counts.append(max(outputs))
    else:
        counts.append(0)
    #print(count, max(outputs))
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count))


1780it [05:27,  5.43it/s]


In [ ]:
from PIL import Image, ImageFile
from deep_sort.deep_sort.ocsort import OCSort
from deep_sort.application_util.preprocessing import non_max_suppression

ImageFile.LOAD_TRUNCATED_IMAGES = True
file = open('submission_oc_det0.5_suppress.txt', 'w')
file.write("Id,Predicted")
counts = []
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    #print("######################### NEW SEQ ##################")
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    tracker = OCSort(det_thresh=0.5, max_age=30, min_hits=3, 
        iou_threshold=0, delta_t=3, asso_func="iou", inertia=0.2, use_byte=False)
    #results = []
    for index, img_row in img_rows.iterrows():
        image = Image.open(TRAIN_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        boxes = np.array([d["bbox"] for d in detections])
        scores = np.array([d["conf"] for d in detections])
        indices = non_max_suppression(boxes, 0.9, scores)
        detections = [detections[i] for i in indices]
        dets = np.zeros((1, 4))
        if detections:
            dets = np.array([d["bbox"] for d in detections])
        cates = np.array([d["category"] for d in detections])
        confs = np.array([d["conf"] for d in detections])
        output = tracker.update_public(dets, cates, confs)
        if not output.size == 0:
            outputs.append(output[:,4].astype('float64').max())
    #count = len(tracker.trackers)
    if not len(outputs) == 0:
        counts.append(max(outputs))
    else:
        counts.append(0)
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count))
file.close()

1051it [03:19,  6.95it/s]

In [3]:
from PIL import Image, ImageFile
from deep_sort.deep_sort.ocsort import OCSort

ImageFile.LOAD_TRUNCATED_IMAGES = True
file = open('submission_oc_det0.8.txt', 'w')
file.write("Id,Predicted")
counts = []
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    #print("######################### NEW SEQ ##################")
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    tracker = OCSort(det_thresh=0.8, max_age=30, min_hits=3, 
        iou_threshold=0, delta_t=3, asso_func="iou", inertia=0.2, use_byte=False)
    results = []
    for index, img_row in img_rows.iterrows():
        image = Image.open(TRAIN_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        dets = np.zeros((1, 4))
        if detections:
            dets = np.array([d["bbox"] for d in detections])
        cates = np.array([d["category"] for d in detections])
        confs = np.array([d["conf"] for d in detections])
        output = tracker.update_public(dets, cates, confs)
        #print("return", output)
    count = len(tracker.trackers)
    counts.append(count)
    file.write('\n' + str(seq_row["seq_id"]) + ',' + str(count))
file.close()

0it [00:00, ?it/s]/home/briancy2/iwildcam2022/deep_sort/deep_sort/ocsort.py:41: RuntimeWarning: invalid value encountered in sqrt
  w = np.sqrt(x[2] * x[3])
114it [00:40,  3.34it/s]/home/briancy2/iwildcam2022/deep_sort/deep_sort/ocsort.py:42: RuntimeWarning: invalid value encountered in true_divide
  h = x[2] / w
1162it [04:13,  9.20it/s]/home/briancy2/iwildcam2022/deep_sort/deep_sort/association.py:19: RuntimeWarning: invalid value encountered in true_divide
  o = wh / ((bboxes1[..., 2] - bboxes1[..., 0]) * (bboxes1[..., 3] - bboxes1[..., 1])
1780it [06:20,  4.67it/s]


In [33]:
#ImageFile.LOAD_TRUNCATED_IMAGES = True
#file = open('submission_mdv40.95.txt', 'w')
#file.write("Id,Predicted")
results = []
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    base_counts = []
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    for index, img_row in img_rows.iterrows():
        #image = Image.open(TRAIN_DIR+img_row['file_name'])
        detections = df_detection.loc[df_detection.file == ('train/'+img_row['file_name'])]['detections'].to_list()[0]
        l = [d for d in detections if d["conf"] > 0.95]
        count=len(l)
        base_counts.append(count)
    results.append(max(base_counts))

1780it [04:34,  6.48it/s]


In [41]:
loss = np.mean(np.abs((np.array(counts) - train_seq_count["count"].to_numpy())))

In [42]:
print(loss)

3.150561797752809


In [43]:
print(np.asarray(np.unique((np.array(counts) - train_seq_count["count"]), return_counts=True)).T)

[[ -4.   6.]
 [ -3. 176.]
 [ -2.  15.]
 [ -1.  31.]
 [  0.  71.]
 [  1. 156.]
 [  2. 259.]
 [  3. 351.]
 [  4. 307.]
 [  5. 408.]]


In [31]:
loss = np.mean(np.abs(np.array(np.minimum(counts, np.array(results))) - train_seq_count["count"]))

ValueError: operands could not be broadcast together with shapes (1780,) (0,) 

In [32]:
print(loss)

1.4662921348314606


In [25]:
print(np.asarray(np.unique((np.array(np.minimum(counts, np.array(results))) - train_seq_count["count"]), return_counts=True)).T)

[[ -8.   4.]
 [ -7.  15.]
 [ -6.  47.]
 [ -5.  44.]
 [ -4.  52.]
 [ -3.  66.]
 [ -2. 169.]
 [ -1. 399.]
 [  0. 903.]
 [  1.  74.]
 [  2.   7.]]


In [3]:
print(len(counts))

1780


In [26]:
counts_oc = pd.read_csv('submission_oc_det0.5.txt')["Predicted"].to_numpy()
print(np.mean(np.abs((np.array(counts_oc) - train_seq_count["count"].to_numpy()))))
print(np.asarray(np.unique((np.array(counts_oc) - train_seq_count["count"]), return_counts=True)).T)

1.4797752808988764
[[ -8   1]
 [ -7   8]
 [ -6  22]
 [ -5  29]
 [ -4  39]
 [ -3  65]
 [ -2  99]
 [ -1 257]
 [  0 645]
 [  1 331]
 [  2 117]
 [  3  62]
 [  4  24]
 [  5  22]
 [  6  20]
 [  7   8]
 [  8   9]
 [  9   7]
 [ 10   1]
 [ 11   4]
 [ 12   3]
 [ 14   2]
 [ 18   4]
 [ 29   1]]


In [19]:
counts_oc8 = pd.read_csv('submission_oc_det0.8.txt')["Predicted"].to_numpy()
merged_results = np.minimum(counts_oc8, np.array(results))
print(np.mean(np.abs(counts_oc8 - train_seq_count["count"].to_numpy())))
print(np.asarray(np.unique((np.array(counts_oc8) - train_seq_count["count"]), return_counts=True)).T)

1.4578651685393258
[[ -8   3]
 [ -7  16]
 [ -6  28]
 [ -5  40]
 [ -4  55]
 [ -3  60]
 [ -2 117]
 [ -1 356]
 [  0 708]
 [  1 202]
 [  2  77]
 [  3  28]
 [  4  18]
 [  5  13]
 [  6  17]
 [  7  14]
 [  8   8]
 [  9   6]
 [ 10   3]
 [ 12   2]
 [ 14   3]
 [ 16   1]
 [ 17   1]
 [ 18   3]
 [ 23   1]]


In [ ]:
merged_results = np.minimum(counts_oc, np.array(results))
print(np.mean(np.abs(merged_results - train_seq_count["count"].to_numpy())))
print(np.asarray(np.unique((np.array(merged_results) - train_seq_count["count"]), return_counts=True)).T)

In [33]:
print(np.array(results).shape, counts_oc.shape)


(1780,) (1780,)


In [16]:
counts_deep = pd.read_csv('submission_deep_conf_0.5_nn0.txt')["Predicted"].to_numpy()
print(np.mean(np.abs((np.array(counts_deep) - train_seq_count["count"].to_numpy()))))
print(np.asarray(np.unique((np.array(counts_deep) - train_seq_count["count"]), return_counts=True)).T)

1.3780898876404495
[[ -9   3]
 [ -8  20]
 [ -7  18]
 [ -6  36]
 [ -5  41]
 [ -4  58]
 [ -3  96]
 [ -2 167]
 [ -1 432]
 [  0 604]
 [  1 216]
 [  2  63]
 [  3  18]
 [  4   5]
 [  5   1]
 [  6   2]]


In [17]:
merged_deep = np.minimum(counts_deep, np.array(results))
print(np.mean(np.abs(merged_results - train_seq_count["count"].to_numpy())))
print(np.asarray(np.unique((np.array(merged_results) - train_seq_count["count"]), return_counts=True)).T)

1.1258426966292134
[[ -8   3]
 [ -7  21]
 [ -6  48]
 [ -5  47]
 [ -4  55]
 [ -3  69]
 [ -2 184]
 [ -1 432]
 [  0 845]
 [  1  69]
 [  2   7]]


In [40]:
counts_deep0 = pd.read_csv('submission_oc_det0.5.txt')["Predicted"].to_numpy()
print(np.mean(np.abs((np.array(counts_deep0) - train_seq_count["count"].to_numpy()))))
print(np.asarray(np.unique((np.array(counts_deep0) - train_seq_count["count"]), return_counts=True)).T)

1.4797752808988764
[[ -8   1]
 [ -7   8]
 [ -6  22]
 [ -5  29]
 [ -4  39]
 [ -3  65]
 [ -2  99]
 [ -1 257]
 [  0 645]
 [  1 331]
 [  2 117]
 [  3  62]
 [  4  24]
 [  5  22]
 [  6  20]
 [  7   8]
 [  8   9]
 [  9   7]
 [ 10   1]
 [ 11   4]
 [ 12   3]
 [ 14   2]
 [ 18   4]
 [ 29   1]]


In [11]:
loss = np.mean(np.abs((np.array(counts) - train_seq_count["count"].to_numpy())))

In [12]:
print(loss)

1.6578651685393258
